# Task #1
### Load Dataset

Dataset link: https://huggingface.co/datasets/SKNahin/bengali-transliteration-data

In [10]:
from datasets import load_dataset

# load the dataset
ds = load_dataset("SKNahin/bengali-transliteration-data", split='train')
print(ds)

Dataset({
    features: ['bn', 'rm'],
    num_rows: 5006
})


#### Preprocessing

eg: Remove too long or too short sentences

In [ ]:
# Get Minimum & Maximum lengths of the sequences
# min_len, max_len = 0, 0
# for data in ds['rm']:
#     max_len = max(max_len, len(data))
#     min_len = min(min_len, len(data))
# print(min_len, max_len)

min_len, max_len = 1, 200
excluded_idx = []

for i in range(0, len(ds['rm'])):
    sz = len(ds['rm'][i])
    if sz < min_len or sz > max_len:
        excluded_idx.append(i)

print(f"{excluded_idx = }")

dataset = ds.select((
    i for i in range(0, len(ds['rm']))
    if i not in excluded_idx
))

print(f"{dataset = }")

o gula Torrent site e paben
excluded_idx = [136, 385, 532, 540]
dataset = Dataset({
    features: ['bn', 'rm'],
    num_rows: 5002
})


### Do Train Test Split

In [26]:
# split the dataset into train and test
split_data = dataset.train_test_split(test_size=0.2, seed=42)
train_data = split_data['train']
test_data = split_data['test']

# print train-test data metadata
print(f"{train_data = }\n\n{test_data = }")

train_data = Dataset({
    features: ['bn', 'rm'],
    num_rows: 4001
})

test_data = Dataset({
    features: ['bn', 'rm'],
    num_rows: 1001
})
